## Libraries and settings

In [15]:

# Libraries
import os
# os.system('pip install yfinance') to install the yfinance in the current environment
import json
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt
import sqlite3

# Settings
import warnings
warnings.filterwarnings("ignore")

# Current working directory
print(f'Current working directory: {os.getcwd()}')

Current working directory: /workspaces/Scientific-Programming_GroupProject


## Download qoutes (stock information) into one file

In [16]:
# List of stocks (quotes)
stocks = ['AAPL', 'MSFT', 'GOOGL', 'META', 'NFLX']

# Empty list to collect data
all_data = []

# Loop through each stock and collect data
for symbol in stocks:
    data = yf.download(symbol, start='2024-11-01', end='2025-04-01')
    data['Symbol'] = symbol  # Add symbol column
    data = data.reset_index()  # Reset index to make Date a column
    all_data.append(data)  # Append to list

# Combine all dataframes
final_df = pd.concat(all_data, ignore_index=True)


 # Preview the result
print(final_df.head())

# Save the data to a CSV file NEEDED? 
data.to_csv('stocks_data.csv', index=False) 

# Connect to SQLite database (creates 'stocks.db' if it doesn't exist)
conn = sqlite3.connect('stocks.db')

# Save a pandas DataFrame to a table in SQLite
final_df.to_sql('stock_prices', conn, if_exists='append', index=False)
# The file 'stocks.db' is where everything is stored (binary format)

# Read the data back from the SQLite database
df = pd.read_sql_query("SELECT * FROM stock_prices stock_prices LIMIT 10;", conn)
# Close the connection
conn.close() 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Price        Date       Close        High         Low        Open      Volume  \
Ticker                   AAPL        AAPL        AAPL        AAPL        AAPL   
0      2024-11-01  222.420471  224.855115  219.786270  220.484729  65276700.0   
1      2024-11-04  221.522446  222.300732  219.227509  220.504697  44944500.0   
2      2024-11-05  222.959290  223.458192  220.654365  221.312919  28111300.0   
3      2024-11-06  222.230896  225.573545  220.704257  222.121137  54561100.0   
4      2024-11-07  226.980438  227.379569  224.076840  224.136706  42137700.0   

Price  Symbol Close High  Low  ... Close High  Low Open Volume Close High  \
Ticker         MSFT MSFT MSFT  ...  META META META META   META  NFLX NFLX   
0        AAPL   NaN  NaN  NaN  ...   NaN  NaN  NaN  NaN    NaN   NaN  NaN   
1        AAPL   NaN  NaN  NaN  ...   NaN  NaN  NaN  NaN    NaN   NaN  NaN   
2        AAPL   NaN  NaN  NaN  ...   NaN  NaN  NaN  NaN    NaN   NaN  NaN   
3        AAPL   NaN  NaN  NaN  ...   NaN  NaN  

OperationalError: table stock_prices has no column named ('Date', '')